# Introducton

This is our Visual Grounding project, it is structured in the following way:

- Setup of all the necessary libraries and files
- Implementation of the Dataset class used along the whole project
- How we evaluate a model
- Training free-approaches evaluation:
  - Baseline
  - Recursive Region Search
- Setup of required components for training
- Training and evaluation of the models:
  - VG Clip
  - BOH
  - BOH v2
- Conclusion and brief discussion on the work

#Set Up




##Install dependencies

In [ ]:
!pip install git+https://github.com/openai/CLIP.git
!pip install ultralytics

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-lvwm37ca
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-lvwm37ca
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.3 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369499 sha256=d79556b2ada6f483e3d5586baf616a57d93a0895dbb1045ab11b5004652e8b30
  Stored in directory: /tmp/pip-ephem-wheel-cache-dg9mxu1c/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 609.4/609.4 kB 4.3 MB/s eta 0:00:00


##Libraries

In [ ]:
from torch.cuda import is_available
import torchvision
import torch
import os
import pandas as pd
from PIL import Image, ImageDraw
from torchvision.datasets import VisionDataset
from pycocotools.coco import COCO
from torch.utils.data import DataLoader
import torchvision.transforms as T
import torch.nn.functional as F
from google.colab import drive
import numpy as np
import clip
import math
from torchvision.transforms import ToPILImage
from torchvision.transforms.functional import to_tensor
from torch.nn.modules.loss import _Loss
import torch.nn as nn
from torchvision.ops import box_iou, box_convert, complete_box_iou_loss
from torch.nn.functional import softmax, cosine_similarity
import albumentations as A
import albumentations.pytorch as AP
from tqdm import tqdm
from torchvision.ops import box_iou, box_convert
from torch.utils.tensorboard import SummaryWriter

## Dataset and models downloading

In [ ]:
!gdown --id 1xijq32XfEm6FPhUb7RsZYWHc2UuwVkiq
!tar -xvzf refcocog.tar.gz

if not os.path.exists('./refcocog'):
  print("There are some problems loading the dataset")
else:
  print("Dataset downloaded and extracted correctly")

Output streaming troncato alle ultime 5000 righe.
./refcocog/images/COCO_train2014_000000187797.jpg
./refcocog/images/COCO_train2014_000000257782.jpg
./refcocog/images/COCO_train2014_000000340636.jpg
./refcocog/images/COCO_train2014_000000227293.jpg
./refcocog/images/COCO_train2014_000000162435.jpg
./refcocog/images/COCO_train2014_000000049581.jpg
./refcocog/images/COCO_train2014_000000362795.jpg
./refcocog/images/COCO_train2014_000000068439.jpg
./refcocog/images/COCO_train2014_000000421602.jpg
./refcocog/images/COCO_train2014_000000146481.jpg
./refcocog/images/COCO_train2014_000000507939.jpg
./refcocog/images/COCO_train2014_000000476846.jpg
./refcocog/images/COCO_train2014_000000561288.jpg
./refcocog/images/COCO_train2014_000000023287.jpg
./refcocog/images/COCO_train2014_000000132836.jpg
./refcocog/images/COCO_train2014_000000406414.jpg
./refcocog/images/COCO_train2014_000000333355.jpg
./refcocog/images/COCO_train2014_000000189026.jpg
./refcocog/images/COCO_train2014_000000499237.jpg


In [ ]:
%rm -rf runs/


In [ ]:
!gdown --id 1AzRst4lyC9mdb-jH2DIzNvpbpzKETkzg
!unzip -u runs.zip
if not os.path.exists('./runs'):
  print("There are some problems loading the models")
else:
  print("Models downloaded and extracted correctly")

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1AzRst4lyC9mdb-jH2DIzNvpbpzKETkzg
To: /content/runs.zip
100% 325M/325M [00:05<00:00, 55.4MB/s]
Archive:  runs.zip
   creating: runs/
   creating: runs/boh_v2_512/
  inflating: runs/.DS_Store          
  inflating: __MACOSX/runs/._.DS_Store  
   creating: runs/boh_256_8/
   creating: runs/vgclip_256_3_2/
   creating: runs/boh_v2_256_8/
   creating: runs/boh_768_8/
  inflating: runs/boh_v2_512/epoch_39.pth  
  inflating: runs/boh_v2_512/.DS_Store  
  inflating: __MACOSX/runs/boh_v2_512/._.DS_Store  
  inflating: runs/boh_256_8/epoch_20.pth  
  inflating: runs/vgclip_256_3_2/epoch_40.pth  
  inflating: runs/boh_v2_256_8/epoch_28.pth  
  inflating: runs/boh_v2_256_8/.DS_Store  
  inflating: __MACOSX/runs/boh_v2_256_8/._.DS_Sto

#RefCOCOgDataset

In [ ]:
def normalize_bbox(bbox, width, height):
    """
    Normalizes the bounding box coordinates, this function should be
    used only when the bounding boxes are not batched
    :param bbox: original bounding box coordinates
    :param width: width of the image
    :param height: height of the image
    :return: the normalized bounding box coordinates
    """
    return bbox[0] / width, bbox[1] / height, bbox[2] / width, bbox[3] / height


class RefCOCOgDataset(VisionDataset):
    """
    Custom implementation of the RefCOCOg DataSet
    """

    def __init__(self, root, split='train', transform = None, text_transform = None, normalize_bbox=False) -> None:
        img_path = root.rstrip('/') + '/images/'
        ann_path = root.rstrip('/') + '/annotations/instances.json'
        refs_path = root.rstrip('/') + '/annotations/refs(umd).p'

        super().__init__(img_path, transform)

        self.coco = COCO(ann_path)

        refs = pd.read_pickle(refs_path)
        self.refs = [sample for sample in refs if sample['split'] == split]
        self.text_transform = text_transform
        self.transform = transform
        self.normalize_bbox = normalize_bbox

    def _load_image(self, ref: int) -> Image.Image:
        """
        Loads the image for the sample with given id
        :param ref: the id of the sample
        :return: image, width and height
        """
        path = self.coco.loadImgs(ref['image_id'])[0]["file_name"]
        image = Image.open(os.path.join(self.root, path)).convert("RGB")

        # get width and height
        width, height = image.size

        return image, width, height

    def _load_target(self, ref):
        """
        Loads bounding box coordinates for the sample with given id
        :param ref: item of sample
        :param width: the original width of the image
        :param height: the original height of the image
        :return: the bounding box coordinates
        """
        bbox = self.coco.loadAnns(ref['ann_id'])[0]['bbox']

        return bbox

    def _load_text(self, ref):
        """
        Loads the longest referring expression for the sample with given id
        :param ref: item of sample
        :return: the longest referring expression
        """
        text = [sent['sent'] for sent in ref['sentences']]

        if self.text_transform is not None:
            text = self.text_transform(text)

        return text


    def _load_proposals(self, ref):
        """
        Loads the bounding box coordinates for the proposals of the sample with given id
        :param ref: item of sample
        :return: a list of bounding box coordinates
        """
        ann_ids = self.coco.getAnnIds(imgIds=ref['image_id'])
        proposals = self.coco.loadAnns(ann_ids)
        bboxes = [proposal['bbox'] for proposal in proposals]

        return bboxes


    def __getitem__(self, index: int):
        ref = self.refs[index]
        image, width, height = self._load_image(ref)
        text = self._load_text(ref)
        target = self._load_target(ref)
        proposals = self._load_proposals(ref)


        # Apply transformations
        if self.transform:
            # Prepare data for augemntations
            data = {
                'image': np.array(image),
                'bboxes': [target]+proposals,
                'labels': [1] + [0]*len(proposals)
            }

            # Note: transform expects named parameters
            transformed = self.transform(
                image=data['image'],
                bboxes=data['bboxes'],
                labels=data['labels']
            )
            image = transformed["image"]
            bboxes = transformed["bboxes"]
            target, proposals = bboxes[0], [bbox for bbox in bboxes[1:]]

        if self.normalize_bbox:
            target = normalize_bbox(target, width, height)

        return {
            "Image": image,
            "Target": target,
            "Text": text,
            # "Proposals" : proposals
        }

    def __len__(self) -> int:
        return len(self.refs)

In [ ]:
# Initialize the device to use (mps is for Apple neural engine)
DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f'Using device: {DEVICE}')

Using device: cpu


In [ ]:
# Load the YOLOv5 model
yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
clip_model, preprocess = clip.load("RN50", 'cpu')
clip_model = clip_model.to(DEVICE)

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 115.3 MB/s eta

# Evaluation of our models

To evaluate our models, we utilized two metrics. The first one is accuracy (or recall, as they are equivalent for this specific problem). This is computed by considering predictions with an Intersection over Union (IoU) value greater than 0.5 as correctly identified samples, while predictions with IoU values lower than 0.5 are regarded as incorrect.

Additionally, we chose to use the mean IoU as a measure of the model's precision in predicting the bounding boxes. This metric provides an average of how accurately our model delineates the bounding boxes across all test samples, hence giving us a more detailed understanding of its performance.

# Baseline

This is the baseline model implemented as suggested in the assingment, using yolo as our object proposal model and clip for retrieving the proposal more similar to the referring expression.
Since for this method we are evaluating only the cropped image with the given proposal we are loosing the context around the object. For this reason the baseline will not be able to infer spatial and relational informations.
Sentences like "The man on the left" will likely produce wrong predictions.
Other examples can be "The man looking to the red chair" will likely produce wrong predictions if there are other man in the image.

However using yolo that is trained on the RefCOCO datasets, which includes our dataset too, we have almost always perfect predictions and so when the sentences doesn't have spatial or relational references we have a very good prediction with an IoU value close to 1. Otherwise if the model predicts a wrong sample it will be very likely to have a IoU close to zero.
We will use this consideration too when evaluating our models.

The baseline has an overall accuracy on the test set of 53% and a mean IoU of 0.51. As said before the mean in this case is far away than the median values that are around 0 and arround 1

In [ ]:
class Baseline(nn.Module):
    """
    Baseline implementation of the object grounding model.
    Here the first step is to use yolo to extract all the bounding boxes of the objects in the image.
    The bounding box as proposal for are used for object grounding.
    """
    def __init__(self, yolo, clip_model, preprocess):
        super(Baseline, self).__init__()
        self.yolo = yolo
        self.clip_model = clip_model
        self.preprocess = preprocess

    def forward(self, image, sent):
        """
        Forward pass
        :param image: the single image we want to ground (shape: n_channel, height, width)
        :param sent: the referring sentence to an object in the image
        :return the bounding box of the object
        """

        # getting yolo bounding boxes
        yolo_res = self.yolo(image)

        # extract the yolo bounding boxes
        boxes = yolo_res.xyxy[0][:, :4].cpu().numpy()

        if len(boxes) == 0:
            return None


        # get the cropped image for every bounding bbox
        cropped_images = [ image.crop((box[0], box[1], box[2], box[3])) for box in boxes ]
        cropped_images = torch.cat([ self.preprocess(c).unsqueeze(0).float().to(DEVICE) for c in cropped_images ])
        with torch.no_grad():
            # getting the encoding formy sentence
            my_sent_features = clip_model.encode_text(sent.to(DEVICE))
            my_sent_features /= my_sent_features.norm(dim=-1, keepdim=True)

            # getting the encoding for the cropped images
            image_features = clip_model.encode_image(cropped_images)
            image_features /= image_features.norm(dim=-1, keepdim=True)

            # compute the similarity between the query sentence and the engineered sentences
            similarity = (100.0 * my_sent_features @ image_features.T).softmax(dim=-1)

            # get the index of the maximum similarity value
            max_similarity = torch.max(similarity, dim=1)
            global_max_index = torch.argmax(max_similarity.values)
            index = max_similarity.indices[global_max_index]

            # get the bbox of the sentence
            return boxes[index]

In [ ]:
# Load the test dataset for yolo, we want to get only the first sentence tokenized
def get_first(sents): return clip.tokenize(sents[0]).to(DEVICE)

test_dataset = RefCOCOgDataset(root='./refcocog/', split='test', text_transform=get_first)

loading annotations into memory...
Done (t=22.33s)
creating index...
index created!


In [ ]:
# evaluate the test set
def evaluate_one_by_one(model, dataset):
    """
    With this function we evaluate models that are suited to work with only one
    sample at a time, for example the baseline where we have to use yol
    :return:
    """

    mIoU = 0
    true_positive_iou = 0
    false_negative_iou = 0

    for i, sample in tqdm(enumerate(dataset)):
        sent = sample["Text"]
        image = sample["Image"]
        bbox = model(image, sent)

        # if no bbox predicted (caused by object proposal failure)
        if bbox is None:
            false_negative_iou += 1
            continue

        # to use the builtin functions we must use tensors
        target = box_convert(torch.tensor(sample["Target"]).unsqueeze(0), in_fmt='xywh', out_fmt='xyxy')
        pred = torch.tensor(bbox).unsqueeze(0)

        # calculate iou
        IoU = box_iou(target, pred)
        if IoU > 0.5: true_positive_iou += 1
        else: false_negative_iou += 1
        mIoU += IoU

    accuracy = true_positive_iou / (true_positive_iou + false_negative_iou)
    mIoU =  mIoU[0].item() / (true_positive_iou + false_negative_iou)
    return accuracy, mIoU

In [ ]:
baseline = Baseline(yolo, clip_model, preprocess)

test_dataset = torch.utils.data.Subset(test_dataset, range(0,200))
acc, miou = evaluate_one_by_one(baseline, test_dataset)

print('Accuracy for IoU:', acc)
print('Mean IoU:', miou)

200it [08:57,  2.69s/it]

Accuracy for IoU: 0.54
Mean IoU: 0.5318921661376953


# Recursive Region Search

This model tries to find the best region by recursuvely cropping the image to yield increasingly better sub-images. This is a training free approach.

### *Divide the image in 3x3 regions*
The initial step is to segment the image into a 3x3 grid. For each region, calculate the similarity to the referring expression. We then apply the softmax function to scale the similarity into a probability distribution.

![picture](https://drive.google.com/uc?id=1TtV9UuSZfqld0pLSK0Dr9Hsjd4BtV_G3)

"*a man about to throw a frisbee*"

### *Put together adjacent regions*

Once we have calculated a score for each region, we try to merge adjacent regions with high similarity scores. To achieve this, we select the region with the highest score and look if any adjacent regions also have high scores.

From the example, we can see that we are combining three regions. We then calculate the weighted sum for every coordinate to identify the new best region. This approach results in the weighted mean region of the image, which is most likely to contain our object.
The weights here are calculated using a second time the softmax, in order to have the sum of weights equals to one.

This process of combining adjacent regions is necessary to predict images with large objects that span more than one region. With a 3x3 grid, we are also able to differentiate between two objects that are far apart from each other.

### *Adding margin to the region*
After identifying the new region of interest, we expand it by adding a margin to avoid losing context. For instance, from the image above, if we don't add a margin to the region, we risk missing part of the object we are searching for, in this case, the legs.


It is important to note that the margin is a hyperparameter. If it is too small, we lose context; if it's too large, the algorithm will slow down due to the smaller crop applied at every iteration leading to an increase in the number of evaluations. We found that adding a 50% margin is a reasonable compromise.


### *Recursively repeat with the new region*
Once we have the newly expanded region, we recompute the similarity for the entire region. If the similarity score is lower than that of the previous region, it implies we might be focusing on regions that are too small, and so we use this as our termination criterion.

![picture](https://drive.google.com/uc?id=11uDHN00laLMlFMYM5tu9S-zYqPf8RxTi)
![picture](https://drive.google.com/uc?id=1WL4Whnhg6k66PCd0Mv74ENJux0ey281H)


As illustrated here, the algorithm sometimes identifies two different regions of interest because they are separated by a region with very small probability. We have set a threshold, whereby regions with probabilities lower than 0.1 are disregarded. In this case, we proceed by applying the margin and computing the similarity for both regions, and continue the recursive process on the region with the highest similarity.

![picture](https://drive.google.com/uc?id=1e1A3aT4UEkMKe9dw-G45pOk9NBoJqVH9)

In the image above, we display the final stage of recursion. After this point, the algorithm identifies two smaller regions with lower similarity than the original one, at which point it stops. The final result is depicted below. The ground truth is highlighted in green, and our prediction is in red.

![picture](https://drive.google.com/uc?id=1CfFWl8fcBFzx3W3Zs9A4HiJxgh0931w9)


### *Conclusion*

This method has several limitations. As evidenced by the example above, it is challenging for the algorithm to accurately encapsulate the object within the bounding box. However, it is generally able to locate the correct part of the image where the object is situated. While this method does not outperform the baseline, it shares similar limitations for example in understanding spatial references.
Another problem of this model is the difficulty to understand the correct shape of the bounding box, as we can see from the example the prediction cuts the head of the man. Even for the other samples where it can find a correct prediction we can see the same problem.

The overall accuracy on the test set is of 11% and the mean IoU is 0.25.

In [ ]:
class RecursiveRegionModel(nn.Module):
    """
    This model divides the image in a 3x3 grid and recursively selects the
    most promising region until the similarity starts to drop.
    Moreover, at every step the most promising regions are put together if
    they are adjacent and the new region is taken considering the similarity
    contribution, so if we have two adjacent regions, one with weight 0.7
    and the other one of 0.3 the new region will be more shifted to the first one.
    To handle cases where the referring expression refers to an object occurring
    multiple times, we do not consider recursively one region but we want to conisder
    up to 2 new regions at every iteration.
    """

    def __init__(self, clip_model, preprocess, device):
        super(RecursiveRegionModel, self).__init__()
        self.clip_model = clip_model
        self.preprocess = preprocess
        self.device = device
        # adjacency mapping for the regions
        self.adj_mapping = {
            0: [1, 3],
            1: [0, 2, 4],
            2: [1, 5],
            3: [0, 4, 6],
            4: [1, 3, 5, 7],
            5: [2, 4, 8],
            6: [3, 7],
            7: [4, 6, 8],
            8: [5, 7]
        }

    def forward(self, image, sent):
        sent = self.clip_model.encode_text(sent).to(self.device)
        return self.recursive_region_search(image, sent)

    def recursive_region_search(self, image, ref_exp_emb, threshold=0.0):
        """
        Divide the image in 9 regions, combine good adjacent regions and recursively search for the object
        :param image: the image to ground, it should be in PIL format
        :param ref_exp_emb: the referring expression of the object in the image, in the form of a CLIP embedding
        :param threshold: the threshold of similarity to consider a region good
        :param depth: max depth of the recursion
        :return:
        """
        width, height = image.size

        # if the image is too small we return None
        if width < 5 or height < 5:
            return None

        # grid 3x3 boxes to divide the image
        boxes = [
            (0, 0, width // 3, height // 3),                                    # Top left
            (width // 3, 0, (width // 3) * 2, height // 3),                     # Top center
            ((width // 3) * 2, 0, width, height // 3),                          # Top right
            (0, height // 3, width // 3, (height // 3) * 2),                    # Middle left
            (width // 3, height // 3, (width // 3) * 2, (height // 3) * 2),     # Middle center
            ((width // 3) * 2, height // 3, width, (height // 3) * 2),          # Middle right
            (0, (height // 3) * 2, width // 3, height),                         # Bottom left
            (width // 3, (height // 3) * 2, (width // 3) * 2, height),          # Bottom center
            ((width // 3) * 2, (height // 3) * 2, width, height),               # Bottom right
        ]

        # Crop the image in 9 regions
        regions = [image.crop(box) for box in boxes]

        # Calculate similarity on the 9 regions
        similarities = []
        for index, region in enumerate(regions):
            tensor_img = self.preprocess(region).unsqueeze(0).float().to(self.device)
            # Generate the CLIP embedding for the region
            region_emb = self.clip_model.encode_image(tensor_img)
            similarity = ref_exp_emb @ region_emb.T
            # Compute the cosine similarity between the referring expression and the region
            similarities.append(similarity[0][0])

        similarities = torch.tensor(similarities)

        # Combine adjacent regions with the highest similarity
        bboxes = self.combine_regions(image, boxes, similarities)

        # similarity on the combined regions
        matches = []
        for i, bbox in enumerate(bboxes):
            cropped_img = image.crop(bbox)
            # Compute the cosine similarity between the referring expression and the combined region
            tensor_img = self.preprocess(cropped_img).unsqueeze(0).float().to(self.device)
            combined_emb = self.clip_model.encode_image(tensor_img)
            similarity = ref_exp_emb @ combined_emb.T
            matches.append((cropped_img, similarity, bbox))

        # get the best from the matches
        img, similarity, bbox = max(matches, key=lambda x: x[1])

        # If the similarity is below the threshold, return the current region
        if similarity < threshold:
            return None

        # Otherwise, recursively search the region with the highest similarity
        res_bbox = self.recursive_region_search(img, ref_exp_emb, similarity)

        if res_bbox is None:
            # if the region is too small, return the current region
            return bbox
        else:
            # upscaling bbox -> during recursion the created grid is relative to the cropped image
            # as the recursion goes deeper, the grid is smaller and smaller, and when coming back from
            # the recursion, the grid is upscaled to the original image adding x1 and y1 to the bbox
            # left, upper, right, and lower
            bbox[2] = bbox[0] + res_bbox[2]
            bbox[3] = bbox[1] + res_bbox[3]
            bbox[0] = bbox[0] + res_bbox[0]
            bbox[1] = bbox[1] + res_bbox[1]
            return bbox

    def combine_regions(self, image, regions, similarities):
        """
        Combine adjacent regions with the highest similarity
        :param image: the original image
        :param regions: the regions of the image to evaluate and combine
        :param similarities: the similarities of the regions to the referring expression
        :return: a set of bounding boxes relative to the similar regions
        """

        # Compute weights for each region based on their similarity scores
        weights = similarities.softmax(dim=-1)

        # get an ordered list of the indices similarities, highest to lowest
        max_sim_idxs = torch.argsort(similarities, descending=True)

        # regions with too low similarity are discarded
        filtered_idx = []
        for i in max_sim_idxs:
            if weights[i] > 0.1:
                filtered_idx.append(i)

        # if empty take only the first
        if len(filtered_idx) == 0:
            filtered_idx.append(max_sim_idxs[0])

        # while indexes are adjacent, combine them weighted by their similarity
        combined_regions = []

        max_adj_concat = 0
        # While indexes are adjacent, combine them weighted by their similarity
        while len(filtered_idx) > 0:
            # do not consider more than 2 macro regions
            if max_adj_concat >= 2:
                break

            idx = filtered_idx.pop(0)
            # if there is only one region left, return that region
            if not filtered_idx or len(filtered_idx) == 0:
                combined_region = [regions[idx][0], regions[idx][1], regions[idx][2], regions[idx][3]]
                combined_region = self.add_margin(image, combined_region, 0.5)
                combined_regions.append(combined_region)
                break

            # add adjacency
            idxs = [idx.item()]
            while len(filtered_idx) > 0:
                added = False
                for i in idxs:
                    # patience tells us how many times we can skip a region not adjacent to the current one
                    # this is to allow that for example the third region (ordered by similarity) is added
                    # to the adjacency of the first region, even if the second region is not adjacent to the first
                    patience = 0
                    # if any of filtered_idx is in adj_mapping[i] add it
                    for rm, j in enumerate(filtered_idx):
                        if j in self.adj_mapping[i]:
                            idxs.append(filtered_idx.pop(rm).item())
                            added = True
                        elif patience < 1:
                            patience += 1
                        else:
                            break
                if not added:
                    break

            # left, upper, right, and lower
            combined_region = [0, 0, 0, 0]

            new_weights = weights[idxs]
            new_weights /= 0.01
            new_weights = new_weights.softmax(dim=-1)

            for idx, i in enumerate(idxs):
                combined_region[0] += regions[i][0] * new_weights[idx].item()
                combined_region[1] += regions[i][1] * new_weights[idx].item()
                combined_region[2] += regions[i][2] * new_weights[idx].item()
                combined_region[3] += regions[i][3] * new_weights[idx].item()

            # add 25% margin on each side
            combined_region = self.add_margin(image, combined_region, 0.5)

            combined_regions.append(combined_region)
            max_adj_concat += 1

        return combined_regions

    def add_margin(self, image, bbox, margin):
        """
        Adds the margin to the new region in order to capture more context and
        :param image: image
        :param bbox: region bounding box
        :param margin: percentage of margin to apply
        :return:
        """
        # left, upper, right, and lower
        bbox[0] = max(0, bbox[0] - bbox[0] * margin)
        bbox[1] = max(0, bbox[1] - bbox[1] * margin)
        bbox[2] = min(image.size[0], bbox[2] + bbox[2] * margin)
        bbox[3] = min(image.size[1], bbox[3] + bbox[3] * margin)
        return bbox


In [ ]:
model = RecursiveRegionModel(clip_model, preprocess, DEVICE)

acc, miou = evaluate_one_by_one(model, test_dataset)

print('Accuracy for uoi:', acc)
print('Mean IoU:', miou)

2391it [17:43,  1.67it/s]

# Loss Function

In our model development process that involved training, we had to decide on the appropriate loss function to use. We discovered that in the literature pertaining to bounding box regression, differentiable Intersection over Union (IoU) functions are commonly used. Utilizing this kind of loss function allows the model to optimize the metric that we use for evaluation, which is IoU (note that IoU itself is not differentiable and is needed to calculate accuracy).

We found that most of the time, in the literature, Complete IoU is used. There is even a PyTorch implementation for this. However, we encountered an issue with the Apple Neural Engine; when we tried to train a model using this loss function, Python would crash.

Consequently, we looked for other solutions and came across Scylla Loss from the paper, [SIoU Loss: More Powerful Learning for Bounding Box Regression](https://arxiv.org/pdf/2205.12740.pdf).

According to the paper, this loss function is more efficient than Complete IoU. Since a built-in implementation of Scylla Loss does not exist in PyTorch, we implement it ourselves.

In simple terms, Scylla Loss adds costs for incorrect shape, distance, and angle (which can be viewed as an alignment factor). We won't delve into how this loss function works in detail, as it is comprehensively described in the referenced paper.


### *Modifications*

In the original paper, the authors used the following formula to compute the angle cost:
$$ 1-2\cdot\sin\left(\arcsin\left(x\right)\ -\ \frac{\pi}{4}\right)^{2} $$

However, this equation has a quite complex derivative, so we tried to see with Wolfram Alpha if we could simplify the function. We discovered that the following function was simpler and also had a more straightforward derivative:

$$ {2x}\cdot\sqrt({1}-\ x^{2}) $$

With this modification, the computation of the loss became significantly faster than before. Naturally, we double-checked that this simplification was correct. We checked for other possible simplifications but this was the only one possible.

Another modification we introduced was in the calculation of the absolute values. Instead of using `torch.abs(a - b)`, we used `torch.maximum(a, b) - torch.minimum(a, b)`. We made this change because PyTorch hadn't implemented the absolute value function for the Neural Engine, and it defaulted back its calculation to the CPU, thereby significantly slowing down the training process.

## Scylla Loss Implementation

Here we have the implementation of the Scylla loss from the paper.

In [ ]:
class SIoULoss(_Loss):
    """
    Scylla Loss from the paper "SIoU Loss: More Powerful Learning for Bounding Box Regression"
    It has state-of-the-art performance on object detection tasks
    """

    def __init__(self):
        super(SIoULoss, self).__init__()

    def get_center(self, bbox):
        cx = (bbox[:, 0] * 2 + bbox[:, 2]) / 2
        cy = (bbox[:, 1] * 2 + bbox[:, 3]) / 2
        return cx, cy

    def iou(self, pred, target):
        """
        Calculate the Intersection over Union (IoU) of batched bounding boxes.
        :param pred: predicted bounding boxes (x, y, w, h)
        :param target: target bounding boxes (x, y, w, h)
        """

        # Calculate the (x, y)-coordinates of the intersection rectangle
        x_inter = torch.maximum(pred[:, 0], target[:, 0])
        y_inter = torch.maximum(pred[:, 1], target[:, 1])
        w_inter = torch.minimum(pred[:, 0] + pred[:, 2], target[:, 0] + target[:, 2]) - x_inter
        h_inter = torch.minimum(pred[:, 1] + pred[:, 3], target[:, 1] + target[:, 3]) - y_inter

        # Check for no overlap condition
        no_overlap = torch.where((w_inter < 0) | (h_inter < 0))

        # Calculate the area of intersection rectangle
        inter_area = w_inter * h_inter
        inter_area[no_overlap] = 0.0

        # Calculate the area of both prediction and ground-truth rectangles
        pred_area = pred[:, 2] * pred[:, 3]
        target_area = target[:, 2] * target[:, 3]

        # Calculate the union area by using the formula: Union(A, B) = A + B - Inter(A, B)
        union_area = pred_area + target_area - inter_area

        # Compute the IoU
        iou = inter_area / union_area
        return iou

    def angle_cost(self, pred, target):
        """
        Calculate the angle cost of batched bounding boxes.
        :param pred: predicted bounding boxes (x, y, w, h)
        :param target: target bounding boxes (x, y, w, h)
        """
        # getting the center of the bounding boxes (considering [x, y, w, h] format)
        b_cx, b_cy = self.get_center(pred)
        b_cx_gt, b_cy_gt = self.get_center(target)

        # distance along y of the two centers
        ch = torch.maximum(b_cy_gt, b_cy) - torch.minimum(b_cy_gt, b_cy)

        # euclidean distance between the two centers
        sigma = torch.sqrt((b_cx_gt - b_cx) ** 2 + (b_cy_gt - b_cy) ** 2)
        epsilon = 0.00001

        x = ch / (sigma + epsilon)

        # _lambda = torch.sin(2 * torch.asin(x))
        # less complex function than _lambda (by wolfram alpha)
        return 2 * x * torch.sqrt(1 - x ** 2)

    def distance_cost(self, pred, target, gamma):
        """
        Calculate the distance cost of batched bounding boxes.
        :param pred: predicted bounding boxes (x, y, w, h)
        :param target: target bounding boxes (x, y, w, h)
        :param gamma: this is a value depending on the angle cost (2 - angle_cost)
        """
        b_cx, b_cy = self.get_center(pred)
        b_cx_gt, b_cy_gt = self.get_center(target)

        # maximum distance along the most distant point of the two boxes on x axes
        cw = torch.maximum(pred[:, 0] + pred[:, 2], target[:, 0] + target[:, 2]) - torch.minimum(pred[:, 0], target[:, 0])
        # maximum distance along the most distant point of the two boxes on y axes
        ch = torch.maximum(pred[:, 1] + pred[:, 3], target[:, 1] + target[:, 3]) - torch.minimum(pred[:, 1], target[:, 1])

        px = ((b_cx_gt - b_cx) / cw) ** 2
        py = ((b_cy_gt - b_cy) / ch) ** 2

        return 2 - torch.exp(-1 * gamma * px) - torch.exp(-1 * gamma * py)

    def shape_cost(self, pred, target, theta):
        """
        Calculate the shape cost of batched bounding boxes.
        :param pred: predicted bounding boxes (x, y, w, h)
        :param target: target bounding boxes (x, y, w, h)
        :param theta: this is a hyperparameter, it should vary between 2 and 6 (from the paper)
        """
        # not using the torch.abs function because it is not supported by mps backend

        # calculate shape ratio for width
        omega_w = torch.maximum(pred[:, 2], target[:, 2]) - torch.minimum(pred[:, 2], target[:, 2])
        omega_w = omega_w / torch.maximum(pred[:, 2], target[:, 2])

        # calculate shape ratio for height
        omega_h = torch.maximum(pred[:, 3], target[:, 3]) - torch.minimum(pred[:, 3], target[:, 3])
        omega_h = omega_h / torch.maximum(pred[:, 3], target[:, 3])

        return (1 - torch.exp(-omega_h)) ** theta + (1 - torch.exp(-omega_w)) ** theta

    def forward(self, pred, target):
        """
        Forward pass of the loss
        :param pred: predicted relative bounding boxes (x, y, w, h)
        :param target: target relative bounding boxes (x, y, w, h)
        :return: the SIoU loss
        """
        # compute the intersection over union
        iou = self.iou(pred, target)

        # compute the angle cost
        angle_cost = self.angle_cost(pred, target)

        # compute distance
        distance_cost = self.distance_cost(pred, target, 2 - angle_cost)

        # compute shape cost
        shape_cost = self.shape_cost(pred, target, 4)

        # put everything together
        loss = 1 - iou + distance_cost / 2 + shape_cost / 2
        return loss.sum()

## Complete IoU Loss Implementation

Here we have the implementation of the Complete IoU loss, where we just convert the format of the bounding boxex because the built in implementation requires the `xyxy` format.

In [ ]:
class CIoULoss(_Loss):
    """
    Complete IoU loss implementation using the format xywh.
    """
    def __init__(self):
        super(CIoULoss, self).__init__()

    def forward(self, pred, target):
        # pred and target should be in [x, y, w, h] format
        pred = box_convert(pred, in_fmt='xywh', out_fmt='xyxy')
        target = box_convert(target, in_fmt='xywh', out_fmt='xyxy')
        return complete_box_iou_loss(pred, target, reduction='mean')


# Dataloader and augmentations


### *Images and bounding boxes*

Here, we have defined our dataloader for the training phase. We used the `albumentations` library to perform data augmentation on images with bounding boxes. Our goal was to implement as many types of augmentations as possible where the corresponding transformation could be applied to the bounding box. For instance, we did not apply any rotation to the images because it would be impossible to similarly rotate the bounding box.


### *Text*

Given that we typically have more than one sentence for each image, we needed to find a strategy to use all of them during training. Our approach was to randomly select one sentence; this random selection can be seen as a form of data augmentation. In subsequent epochs, the same image will be associated with a different referring expression. By adopting this strategy, we also eliminated the complexity of handling multiple sentences during training.


### *Dataloaders*

We divided the training data into validation and training sets, allocating 20% of the data for validation and the remaining 80% for training.

After splitting the data, we created our dataloaders (with shuffled data for training). We selected a batch size of 64 for the virtual machine and Google Colab. For the Apple Neural Engine, I increased the size to 96 in order to have a better balance between training time and available memory.

In [ ]:
# augmentation on the image using albumentations
transform_augmented = A.Compose([
    A.CropAndPad(px=None, percent=[0.00,0.1], pad_mode=0, pad_cval=0, pad_cval_mask=0, keep_size=True, sample_independently=True, interpolation=1, always_apply=False, p=0.5),
    A.GaussianBlur(blur_limit=(1, 3), sigma_limit=0, always_apply=False, p=0.1),
    A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), always_apply=False, p=0.1),
    A.Equalize(mode='cv', by_channels=True, mask=None, mask_params=(), always_apply=False, p=0.1),
    A.FancyPCA(alpha=0.1, always_apply=False, p=0.1),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, always_apply=False, p=0.1),
    A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), always_apply=False, p=0.1),
    A.JpegCompression(quality_lower=99, quality_upper=100, always_apply=False, p=0.1),
    A.PixelDropout(dropout_prob=0.01, per_channel=False, drop_value=0, mask_drop_value=None, always_apply=False, p=0.1),
    A.Posterize(num_bits=4, always_apply=False, p=0.1),
    A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), always_apply=False, p=0.1),
    A.Resize(224, 224, interpolation=1),  # Bicubic interpolation
    A.CenterCrop(224, 224),
    A.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711]),
    AP.transforms.ToTensorV2()
], bbox_params=A.BboxParams(format='coco', label_fields=['labels']))


# this is the transformation for the test set, it is only a copy of the preprocess
# of clip using albumentation so we can replace it in our dataset
transform_test = A.Compose([
    A.Resize(224, 224, interpolation=1),  # Bicubic interpolation
    A.CenterCrop(224, 224),
    A.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711]),
    AP.transforms.ToTensorV2()
], bbox_params=A.BboxParams(format='coco', label_fields=['labels']))



def transform_text(sents):
    """
    Get random sentence from the list of sentences,
    we can see this as a form of data augmentation
    :param sents: list of sentences
    :return:
    """
    sent = sents[torch.randint(len(sents), (1,)).item()]
    return sent

# create the datasets for training and testing
train = RefCOCOgDataset(root='./refcocog/', split='train', normalize_bbox=True, transform=transform_augmented, text_transform=transform_text)
test = RefCOCOgDataset(root='./refcocog/', split='test', normalize_bbox=True, transform=transform_test, text_transform=transform_text)

# NOTE: the following line of code is a very important error:
# we are splitting the training dataset into validation and test when
# we already have a validation split.
# We add here the corrected lines of code, but we keep them commented
# in order to be able to reproduce our results.
# Unfortunately we found this error after all models were trained and we did
# not had time to train again any model.
# validation = RefCOCOgDataset(root='./refcocog/', split='val', normalize_bbox=True, transform=transform_test, text_transform=transform_text)
# val_dataloader = DataLoader(validation, batch_size=64, shuffle=False, num_workers=0)

# We discuss better this problem on the conclusion chapter


# split the training dataset into train and validation, 20% of the data will be used for validation
train_dataset, validation_dataset = torch.utils.data.random_split(train, [math.floor(len(train) * 0.8), math.ceil(len(train) * 0.2)])

# create the dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
val_dataloader = DataLoader(validation_dataset, batch_size=64, shuffle=False, num_workers=0)
test_dataloader = DataLoader(test, batch_size=64, shuffle=False, num_workers=0)

/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:778: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(


loading annotations into memory...
Done (t=4.50s)
creating index...
index created!
loading annotations into memory...
Done (t=4.42s)
creating index...
index created!


# Setup for training and evaluation

In the following code we implemented the base functions that we will use across all our models.

We followed the training principles used in the laboratories of this course, we logged to tensorboard the loss, the accuracy and the mean IoU.

Moreover, we are saving our models at every epoch, in that way we are creating checkpoints to be able to stop and resume the training without loosing previous epochs.

In [ ]:
def training_step(net, data_loader, optimizer, cost_function, tokenize, device='cpu'):
    """
    Training step for the network over one single epoch
    """
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_acc = 0.0
    cumulative_iou = 0.0

    # set the network to training mode
    net.train()

    bar = tqdm(data_loader)

    # iterate over the training set
    for sample in bar:
        # load data into DEVICE
        image = sample['Image'].to(device)
        text = clip.tokenize(sample['Text']).to(device)
        bbox = torch.stack(sample['Target']).transpose(0, 1).float().to(device)
        # forward pass
        outputs = net(image, text)
        # loss computation
        loss = cost_function(outputs, bbox)

        # backward pass
        loss.backward()

        # parameters update
        optimizer.step()

        # gradients reset
        optimizer.zero_grad()

        # fetch prediction and loss value
        samples += image.shape[0]
        cumulative_loss += loss.item()

        pred = box_convert(outputs.detach().cpu(), in_fmt='xywh', out_fmt='xyxy')
        target = box_convert(bbox.detach().cpu(), in_fmt='xywh', out_fmt='xyxy')
        iou = box_iou(pred, target).diag()

        # get num of correct predictions -> iou > 0.5
        correct = (iou > 0.5).sum().item()

        cumulative_iou += iou.sum().item()
        cumulative_acc += correct

        # print current loss on progress bar
        bar.set_description("Loss: {}, Acc: {}, mIoU: {}".format(
            loss.item() / len(bbox),
            correct / len(bbox),
            iou.mean().item()
        ))

    return cumulative_loss / samples, cumulative_acc / samples, cumulative_iou / samples

def test_step(net, data_loader, cost_function, tokenize, device='cpu'):
    """
    Test step for the network
    """
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_acc = 0.0
    cumulative_iou = 0.0

    # set the network to evaluation mode
    net.eval()

    bar = tqdm(data_loader)

    # disable gradient computation (we are only testing, we do not want our model to be modified in this step!)
    with torch.no_grad():
        # iterate over the test set
        for sample in bar:
            # load data into GPU
            image = sample['Image'].to(device)
            text = clip.tokenize(sample['Text']).to(device)
            bbox = torch.stack(sample['Target']).transpose(0, 1).float().to(device)

            # forward pass
            outputs = net(image, text)

            # loss computation
            loss = cost_function(outputs, bbox)

            # fetch prediction and loss value
            samples += image.shape[0]
            cumulative_loss += loss.item()

            pred = box_convert(outputs.detach().cpu(), in_fmt='xywh', out_fmt='xyxy')
            target = box_convert(bbox.detach().cpu(), in_fmt='xywh', out_fmt='xyxy')
            iou = box_iou(pred, target).diag()

            # get num of correct predictions -> iou > 0.5
            correct = (iou > 0.5).sum().item()

            cumulative_iou += iou.sum().item()
            cumulative_acc += correct

            # print current loss on progress bar
            bar.set_description("Loss: {}, Acc: {}, mIoU: {}".format(
                loss.item() / len(bbox),
                correct / len(bbox),
                iou.mean().item()
            ))

    return cumulative_loss / samples, cumulative_acc / samples, cumulative_iou / samples

def load_last_checkpoint(net, exp_name, device):
    """
    Load last checkpoint from the experiment folder
    :param net: network to load the weights into
    :param exp_name: experiment name
    :return: last epoch number, network with loaded weights
    """
    # check if path exists
    if not os.path.exists(f"runs/{exp_name}"):
        os.mkdir(f"runs/{exp_name}")

    # list files in runs/exp1 if not empty
    files = os.listdir(f"runs/{exp_name}")

    # filter starting with epoch_
    files = [f for f in files if f.startswith("epoch_")]
    # extract epoch number
    epochs = [int(f.split("_")[1].split(".")[0]) for f in files]
    # get max
    last_epoch = max(epochs) if len(epochs) > 0 else None

    if last_epoch is not None:
        # load weights
        print("Loading weights for experiment {} from epoch {}".format(exp_name, last_epoch))
        net.load_state_dict(torch.load(
            f"runs/{exp_name}/epoch_{last_epoch}.pth",
            map_location=torch.device(device)
        ), strict=False)
    else:
        last_epoch = 0

    return last_epoch, net


def train_evaluate(net, optimizer, critetion, epochs, experiment_name,
                   train_dataloader, val_dataloader, test_dataloader,
                   tokenize, last_epoch=0, device='cpu'):

    writer = SummaryWriter(log_dir=f"runs/{experiment_name}")

    for e in range(last_epoch, epochs+last_epoch):

        train_loss, train_acc, train_miou = training_step(net, train_dataloader, optimizer, critetion, tokenize, device)
        val_loss, val_acc, val_miou = test_step(net, val_dataloader, critetion, tokenize, device)

        # logs to TensorBoard
        writer.add_scalar(f"train/loss", train_loss, e)
        writer.add_scalar(f"train/acc", train_acc, e)
        writer.add_scalar(f"train/miou", train_miou, e)
        writer.add_scalar(f"validation/loss", val_loss, e)
        writer.add_scalar(f"validation/acc", val_acc, e)
        writer.add_scalar(f"validation/miou", val_miou, e)

        # save only trained parameters
        torch.save({k:v for k,v in net.named_parameters() if v.requires_grad}, f"runs/{experiment_name}/epoch_{e + 1}.pth")

        print('Epoch: {:d}'.format(e+1))
        print('\tTraining loss {:.5f}, acc{:.5f}, mIoU{:.5f}'.format(train_loss, train_acc, train_miou))
        print('\tValidation loss {:.5f}, acc{:.5f}, mIoU{:.5f}'.format(val_loss, val_acc, val_miou))
        print('-----------------------------------------------------')

    print('After training:')
    train_loss, train_acc, train_miou = test_step(net, train_dataloader, critetion, tokenize, device)
    val_loss, val_acc, val_miou = test_step(net, val_dataloader, critetion, tokenize, device)
    test_loss, test_acc, test_miou = test_step(net, test_dataloader, critetion, tokenize, device)

    # log to TensorBoard
    writer.add_scalar(f"train/loss", train_loss, epochs + last_epoch + 1)
    writer.add_scalar(f"validation/loss", val_loss, epochs + last_epoch + 1)
    writer.add_scalar(f"test/loss", test_loss, epochs + last_epoch + 1)

    print('\tTraining loss {:.5f}, Acc: {:.5f}, mIoU: {:.5f}'.format(train_loss, train_acc, train_miou))
    print('\tValidation loss {:.5f}, Acc: {:.5f}, mIoU: {:.5f}'.format(val_loss, val_acc, val_miou))
    print('\tTest loss {:.5f}, Acc: {:.5f}, mIoU: {:.5f}'.format(test_loss, test_acc, test_miou))
    print('-----------------------------------------------------')

    writer.close()

# Visual Grounding CLIP

Before implementing any model, our first step was to review the literature to understand how other researchers have tackled visual grounding. We came across a very recent paper, [RISCLIP: Referring Image Segmentation Framework using CLIP](https://arxiv.org/pdf/2306.08498v1.pdf), where they address the same problem as ours. The main difference lies in their approach, they focus on segmentation, whereas our goal is to predict bounding boxes.

Of course here we need the visual transformer and not the Resnet in order to implement what is inside this paper.


## *Original model explanation*

To briefly explain their approach, they apply a `PreFusion` module, essentially a Backbone Adapter that introduces cross-attention between the textual and visual encoders. Following this, they add a `PostFusion` module on top of the CLIP transformers encoders. These modules are very similart to the PreFusion modules but include Multihead Self-Attention and a Multi-Layer Perceptron at the end.

The next step is to compute the similarity between the attended visual patches from the last PostFusion module and the EOS token. This enables the calculation of similarity between every patch and the EOS token, which is then reshaped to create a saliency map of the grounded object's location.

In their paper, they use a `Refiner` module to upscale this map and generate the segmentation prediction. However, since our aim is to extract bounding boxes, we had to devise a different strategy.
Last but not least, this paper reported state-of-the-art performance on referring expression segmentation.

![picture](https://drive.google.com/uc?id=16ag5jWbCsLccrHPNvKO7sq7oGPTKueWD)

## *Motivations*

Our intention was to use this work just as a starting point, to gain an understanding of how cross-attention operates and how to utilize the intermediate encoder layers of the transformers used by CLIP.

Secondly we found that here we can start to put our first contribution replacing the `Refiner` with something else in order to predict the bounding box


## *Implementation*

We replicated the steps proposed in the paper regarding the `PreFusion` and `PostFusion` adapters. For the final part, we chose to view our saliency map as an image containing activations telling us where the object is expected to be located.
Hence, we added two convolutional layers with a 3x3 kernel, each having 16 and 24 channels respectively. Following this, we flattened the result and used a Multi-Layer Perceptron to generate the bounding box prediction.


The main problem here is that the saliency map has only one value for every visual patch, leading to a 14x14 map that may be too small to have a good prediction. However we decided to not use any upscaling methods and just see how it works, the main focus for our work will be on another kind of architecture.


## *Training results*

To train our models, we utilized a smaller variant than the one described in the original paper. Specifically, our common space dimension was 256, with 3 pre-fusion adapters and 2 post-fusion adapters. We designed the model to be compact so that it could be trained on our machines in a reasonable amount of time. Even so, our model have 18 million parameters.

We initiated training with a learning rate of 1e-04. After 27 epochs, we reduced the learning rate to 1e-05. We executed a total of 40 epochs.

As the graph indicates, we could have continued the training of the model for more epochs since it wasn't overfitting. However, with the limited resources we have, we decided to stop here the training: running 40 epochs took 20 hours. Furthermore, the learning curve was starting to flatten, suggesting that we would never have very big improvements. Hence, for these two reasons, we chose to stop training here.

For reference, the original paper used considerably larger models. Their models used over twice the common space dimension, 6 pre-fusion, and 6 post-fusion adapters. They ran the algorithm for 60 epochs, achieving an accuracy of 70% on segmentation tasks. While we cannot directly compare these numberds to our problem, they provide an indication of how effective this model could potentially be.

![picture](https://drive.google.com/uc?id=1pMxVMvfyi8wlMiUhCHi0JGJShhoz61lp)

Our scaled-down implementation achieved an accuracy of 22% and a mean IoU of 33% on the test set. Considering the early stopping and the model's size, we conclude that these results are more or less satisfactory.

We intend to use this performance as a benchmark for our subsequent model architectures.


## *Model strenghts and weakness*

This model seems to be able to predict very good predictions when the referring expression is not ambiguous. In this example we have the referring expression "a lady pouring wine in a glass":

![picture](https://drive.google.com/uc?id=1vvb54fgJzuvM4Vu7HjOXWSsy6bhbGwRr)

In this case the woman in the bounding box is the only one facing to the camera and the other two are not clearly visible: the first is behind some flowers and the other one is only in the side of the image.

In this second example we can see this model fails to predict the correct object: in the image there are two elephants an the model selected the wrong elephant, but still understood that the main object in the referring expression is an elephant.

"elephant facing towards a red cement wall":

![picture](https://drive.google.com/uc?id=1_aFCsJbl0yVpI5Y0aCP0mrqZMV-9HJWL)








In [ ]:
# for this model we need the vit

clip_model, preprocess = clip.load("ViT-B/16", 'cpu')
clip_model = clip_model.to(DEVICE)

100%|████████████████████████████████████████| 335M/335M [00:02<00:00, 140MiB/s]


In [ ]:
class VGClip(nn.Module):
    """
    VG-Clip model (Visual Grounding Clip), a modification of the original RIS-CLIP architecture
    adapted to Visual Grounding and bbox regression.
    """

    def __init__(
            self,
            clip_model,
            num_pre_fusion_adapters=3,
            num_post_fusion_adapters=2,
            shared_space_dim=128,
            dropout=0.1,
            device='cpu'
    ):
        """
        :param num_pre_fusion_adapters: Number of pre-fusion adapters
        :param shared_space_dim: Shared space dimension
        :param dropout: dropout probability
        """
        super(VGClip, self).__init__()
        self.clip_model = clip_model.requires_grad_(False).to(device)
        self.device = device

        self.pre_fusion_adapters = nn.ModuleList([
            PreFusionAdapter(shared_space_dim=shared_space_dim, dropout=dropout)
            for _ in range(num_pre_fusion_adapters)
        ])

        self.post_fusion_adapters = nn.ModuleList([
            PostFusionAdapter(shared_space_dim=512, dropout=dropout)
            for _ in range(num_post_fusion_adapters)
        ])

        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 24, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Sequential(
            nn.LayerNorm(24 * 14 * 14),
            nn.Linear(24 * 14 * 14, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, 4, bias=False),
            nn.Sigmoid()
        )

    def forward(self, image, text):
        v = self.before_visual_transformer(image)  # [197, batch, 768] -> [CLS + PATCHES, BATCH, HIDDEN_SIZE]
        t = self.before_text_transformer(text)  # [77, batch, 512] -> [SOS + TOKENS + EOS, BATCH, HIDDEN_SIZE]

        num_normal_blocks = 12 - len(self.pre_fusion_adapters)
        # run normal transformer blocks
        for i in range(num_normal_blocks):
            v = self.clip_model.visual.transformer.resblocks[i](v)
            t = self.clip_model.transformer.resblocks[i](t)

        # run pre-fusion adapters
        for i, adapter in enumerate(self.pre_fusion_adapters):
            v_hat, t_hat = adapter(v, t) # calculate the pre fusion step output
            # continue with the normal transformer blocks using the output of
            # the previous step (v an t) and the output of the pre-fusion step (v_hat and t_hat)
            t = self.clip_model.transformer.resblocks[num_normal_blocks + i](t + t_hat)
            v = self.clip_model.visual.transformer.resblocks[num_normal_blocks + i](v + v_hat)

        # get clip final embeddings
        v = self.after_visual_transformer(v).permute(1, 0, 2)
        t = self.after_text_transformer(t).permute(1, 0, 2)

        # run post-fusion adapters
        for adapter in self.post_fusion_adapters:
            v, t = adapter(v, t)

        # remove the CLS token and permute
        v = v[1:, :, :].permute(1, 0, 2)  # [197, batch, 512] -> [batch, 196, 512]

        # get EOS token
        eos = t[0, :, :].unsqueeze(1)  # [batch, 1, 512]

        # compute similarity and create the map
        x = cosine_similarity(eos, v, dim=-1).reshape(-1, 1, 14, 14)  # [batch, 1, 14, 14]

        # regression on the bbox
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # Flatten the tensor

        return self.fc(x)

    def before_visual_transformer(self, x):
        """
        Code copied from the clip implementation in order to preprocess the image ready to be fed into the transformer.
        :param x: image tensor
        :return: embedding tensor for the transformer
        """
        x = self.clip_model.visual.conv1(x)  # shape = [*, width, grid, grid]
        x = x.reshape(x.shape[0], x.shape[1], -1)  # shape = [*, width, grid ** 2]
        x = x.permute(0, 2, 1)  # shape = [*, grid ** 2, width]
        x = torch.cat([
            self.clip_model.visual.class_embedding.to(x.dtype)
            + torch.zeros(x.shape[0], 1, x.shape[-1], dtype=x.dtype, device=x.device),
            x
        ], dim=1)  # shape = [*, grid ** 2 + 1, width]

        x = x + self.clip_model.visual.positional_embedding.to(x.dtype)
        x = self.clip_model.visual.ln_pre(x)
        x = x.permute(1, 0, 2)  # NLD -> LND
        return x

    def after_visual_transformer(self, x):
        """
        Code copied from the clip implementation in order to get the final shared space representation.
        Here we want to get all the patch representations, not only the CLS token.
        :param x: the output of the visual transformer
        :return: the shared space representation of all the patches
        """
        visual = self.clip_model.visual
        x = x.permute(1, 0, 2)  # LND -> NLD
        x = visual.ln_post(x[:, :, :])
        # x = visual.ln_post(x[:, 0, :]) -> this was for the CLS token only

        if visual.proj is not None:
            x = x @ visual.proj
        return x

    def before_text_transformer(self, x):
        """
        Code copied from the clip implementation in order to preprocess the text ready to be fed into the transformer.
        :param x: text tokens tensor
        :return: the embedding tensor for the transformer
        """
        x = self.clip_model.token_embedding(x).type(self.clip_model.dtype)  # [batch_size, n_ctx, d_model]
        x = x + self.clip_model.positional_embedding.type(self.clip_model.dtype)
        x = x.permute(1, 0, 2)  # NLD -> LND
        return x

    def after_text_transformer(self, x):
        """
        Code copied from the clip implementation in order to get the final shared space representation.
        Here we want to get all the token representations, not only the EOS token.
        :param x: the output of the text transformer
        :return:
        """
        x = x.permute(1, 0, 2)  # LND -> NLD
        x = self.clip_model.ln_final(x).type(self.clip_model.dtype)

        # take features from the eot embedding (eot_token is the highest number in each sequence)
        x = x[:, :] @ self.clip_model.text_projection
        # x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)] @ self.clip_model.text_projection
        return x


class PreFusionAdapter(nn.Module):
    """
    Pre-fusion adapter module, this module applies cross attention to both sides of visual and textual features
    """

    def __init__(self, shared_space_dim=128, dropout=0.1):
        super(PreFusionAdapter, self).__init__()
        self.v2s = nn.Linear(768, shared_space_dim)  # transforms the visual features to the shared space
        self.t2s = nn.Linear(512, shared_space_dim)  # transforms the text features to the shared space

        self.ln_v = nn.LayerNorm(shared_space_dim)  # Layer norm for the visual features
        self.ln_t = nn.LayerNorm(shared_space_dim)  # Layer norm for the text features

        self.cross_att_v = nn.MultiheadAttention(shared_space_dim, 8, batch_first=False, dropout=dropout)
        self.cross_att_t = nn.MultiheadAttention(shared_space_dim, 8, batch_first=False, dropout=dropout)

        self.s2v = nn.Linear(shared_space_dim, 768)  # transforms the shared space to the visual space
        self.s2t = nn.Linear(shared_space_dim, 512)  # transforms the shared space to the text space

    def forward(self, v, t):
        # Projection to the shared space
        v, t = self.v2s(v), self.t2s(t)

        # Layer norm, after the shared space projection (should go before but causes problem with mps)
        v, t = self.ln_v(v), self.ln_t(t)

        # Cross attention
        v, t = self.cross_att_v(v, t, t)[0], self.cross_att_t(t, v, v)[0]

        # Projection to the original space
        return self.s2v(v), self.s2t(t)


class PostFusionAdapter(nn.Module):
    """
    Post-fusion adapter module, this module applies cross attention to both sides of visual and textual features
    then self attention on each modality separately and then a MLP like in normal encoder architecture
    """

    def __init__(self, dropout=0.1, shared_space_dim=512, num_heads=8):
        super(PostFusionAdapter, self).__init__()
        # Layer norm, before the cross attention
        self.ln_v1 = nn.LayerNorm(shared_space_dim, elementwise_affine=True)
        self.ln_t1 = nn.LayerNorm(shared_space_dim, elementwise_affine=True)

        # Cross attention on both modalities
        self.cross_att_v = nn.MultiheadAttention(shared_space_dim, num_heads, batch_first=False, dropout=dropout)
        self.cross_att_t = nn.MultiheadAttention(shared_space_dim, num_heads, batch_first=False, dropout=dropout)

        # Layer norm, before the multihead self attention
        self.ln_v2 = nn.LayerNorm(shared_space_dim, elementwise_affine=True)
        self.ln_t2 = nn.LayerNorm(shared_space_dim, elementwise_affine=True)

        # Multihead self attention on each modality separately
        self.mhsa_v = nn.MultiheadAttention(shared_space_dim, num_heads)
        self.mhsa_t = nn.MultiheadAttention(shared_space_dim, num_heads)

        # Layer norm, before the linear
        self.ln_v3 = nn.LayerNorm(shared_space_dim, elementwise_affine=True)
        self.ln_t3 = nn.LayerNorm(shared_space_dim, elementwise_affine=True)

        # Linear layers
        self.linear_v = nn.Sequential(
            nn.Linear(shared_space_dim, shared_space_dim * 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(shared_space_dim * 2, shared_space_dim)
        )
        self.linear_t = nn.Sequential(
            nn.Linear(shared_space_dim, shared_space_dim * 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(shared_space_dim * 2, shared_space_dim)
        )

    def forward(self, v, t):
        # Layer norm, before the cross attention
        v_norm, t_norm = self.ln_v1(v), self.ln_t1(t)

        # Cross attention
        v, t = v + self.cross_att_v(v_norm, t_norm, t_norm)[0], t + self.cross_att_t(t_norm, v_norm, v_norm)[0]

        # Layer norm, before the multihead self attention
        v_norm, t_norm = self.ln_v2(v), self.ln_t2(t)

        # Multihead self attention
        v = v + self.mhsa_v(v_norm, v_norm, v_norm, need_weights=False)[0]
        t = t + self.mhsa_t(t_norm, t_norm, t_norm, need_weights=False)[0]

        v_norm, t_norm = self.ln_v3(v), self.ln_t3(t)

        # Linear
        v, t = v + self.linear_v(v_norm), t + self.linear_t(t_norm)

        return v, t

## Training the model


In [ ]:
net = VGClip(clip_model, num_pre_fusion_adapters=3, num_post_fusion_adapters=2,
             shared_space_dim=256, dropout=0.5,
             device=DEVICE).to(DEVICE)

experiment_name = "vgclip_256_3_2"

last_epoch, net = load_last_checkpoint(net, experiment_name, device=DEVICE)

criterion = SIoULoss()

# Uncomment to train
# optimizer = torch.optim.AdamW(net.parameters(), lr=0.0001, weight_decay=0.02)
# train_evaluate(net, optimizer, criterion, epochs=20,
#               experiment_name=experiment_name,
#               train_dataloader=train_dataloader,
#               val_dataloader=val_dataloader,
#               test_dataloader=test_dataloader,
#               last_epoch=last_epoch,
#               tokenize = clip.tokenize,
#               device=DEVICE)

# Run only the test
test_loss, test_acc, test_miou = test_step(net, test_dataloader, criterion, clip.tokenize, DEVICE)
print('\tTest loss {:.5f}, Acc: {:.5f}, mIoU: {:.5f}'.format(test_loss, test_acc, test_miou))

Loading weights for experiment vgclip_256_3_2 from epoch 40


Loss: 0.7130310304703251, Acc: 0.1935483870967742, mIoU: 0.35104069113731384: 100%|██████████| 79/79 [02:28<00:00,  1.88s/it]

	Test loss 0.71853, Acc: 0.22994, mIoU: 0.34763


# BOH (Bidirectional Optimized Hyperattention) model

Is this a joke? Boh...

## *Introduction*

Reviewing the literature, we noticed that cross-attention is used in almost any work to identify the position of a referred object. This makes sense because cross-attention is one of the most effective ways to share information between different modalities, such as text and images.


Going deeper into how a transformer operates and how it can encode abstract representations, we realized that almost all architectures incorporate one (or more) tokens that capture information during the self-attention pass. For example, in the Visual Transformer, a CLS (Classification) token is used for image classification. Textual encoders have an EOS (End of String) token, and sometimes an SOS (Start of String) token. CLIP also uses these tokens to create its shared space between the visual and textual parts.

## *The idea*

This analysis sparked a question: why not use a token specifically designed to capture the position within an image?

Thus, we decided to explore if a token passed through cross-attention could capture the object's position. We have named this token the LOC (Localization) token.

## *Architecture*

The architecture we decided to implement is very simple. We first introduce the BOH module, which is essentially a cross-attention mechanism. After `N` number of encoder layers of the CLIP transformers, we start to add our BOH model. We perform cross-attention using the textual part as the query and the visual part as the key and value.

Almost all other works we came across applied cross-attention in both ways, using visual as the query and textual as the key and value. We chose to perform cross-attention in one direction only because if the query's purpose in self-attention is to seek something specific, in our case the query is the textual embedding and the answer lies in the visual embedding.

The second reason for this choice was to keep the model as simple as possible. The LOC token is inserted at the first cross-attention module. This token is a parameter whose initial value is trained during the training process.

When we need to feed our BOH module, we have to transform our data to match the dimensions of the two different encoder's embeddings. To achieve this, we added a simple linear layer without any kind of activation, enabling a linear transformation of the data. We refer to this as `shared_space_dim`, which represents the size of all the embeddings of our modules.
We decided to use the same layer with the same weights for all the output of every encoder. So we end up with only two linear transofrmation layer, one for the visual part and one for the textual part. This strategy is commonly used in NLU in order to reduce the number of parameters and to help avoid overfitting (see Weight Tying).

After every cross-attention module, except the last one, we add a residual connection between the output of the previous cross-attention and the output of the respective textual encoder. It should be noted that the output of the cross-attention layer will have one additional token, the LOC token. To perform the residual connection, we exclude the token from the summation and concatenate it back on the token dimension.


Lastly, we feed the LOC token into a Multi-Layer Perceptron (MLP) layer to predict the bounding boxes. It's crucial to mention that we found the MLP layer used in regression should not have a bias. Even with a random model, we observed some good predictions. This led us to conclude that the model, with bias, would only use that to find the mean value of the coordinates of the bounding box and not consider the rest of the input for prediction.


The following scheme clearly illustrates the architecture:

![picture](https://drive.google.com/uc?id=1qiBz5M1obu2wfXuntOpeX15ADrhnXOZC)


## *Implementations*

We opted to train our model using two configurations, using shared spaces of size 768 and 256, respectively. The larger model has approximately 40 million parameters (excluding the CLIP parameters). However, we noticed that using this configuration did not significantly slow down the training. The smaller model has ten times fewer parameters: 4 million. For both models, we utilized the same number of cross-attention modules, set at 8.

## *Training results*

#### *Note on the graphs*
Since it is not possible to compare validation and training loss on tensorboard, we manually extracted the data and used `matplotlib` to create these graphs.

#### *BOH 768*
Let's first examine the training curves for the larger model:
![picture](https://drive.google.com/uc?id=12Z9Yd6FlRCnxFqJ42tBSbehkJXMmBVjx)

Similar to the previous architecture, we stopped our training after 40 epochs, primarily due to limited resources. However, this time, the curve isn't flattening, suggesting that with more epochs, we might obtain better results. Observing the validation curve, we notice some significant steps, primarily occurring when we reduced the learning rate. As before, we started with a learning rate of 1e-04 and gradually decreased it.

On the test set, this model achieved 19.5% accuracy and a mean IoU of 0.30. The accuracy is not too higher than for the previous method. However we cannot conclude anything since we stopped too early the training.

#### *BOH 256*
Now, let's analyze the performance of the smaller model. As we can see, the learning curve doesn't show improvement after 20 epochs, leading us to stop the training at this point and to choose the model at epoch 20 as the best model.
![picture](https://drive.google.com/uc?id=1wpjnNWNHYZ5Szn80_VmrtbfN2qj0nW9A)

On the test set this model achieved 18% accuracy and 0.299 mIoU. The accuracy is still higher than the VGClip model and not far away from the bigger model.
However we can infer looking at the learning curve is that this is the maximum performance that we can achieve with such a small model and it is very likely that with more epochs we are not getting any better.


#### *Comparison*
Here we can see a better comparison between the curve of the two models where the purple curve is the bigger model and the gray one the smaller one:
![picture](https://drive.google.com/uc?id=1w3bu1do12gb5p3nlpRDtn97z0aXcYXOH)



Here we can see the two samples of the elephant, the first is with the big model and the second one with the smallest. As we can see there is not much difference.

<img src="https://drive.google.com/uc?id=1qEzJfSt3V82I4ipWlD21wCuhgYLpBbFT" width="500"> <img src="https://drive.google.com/uc?id=1js36KjKYO64cQiKM84Nyz-8aqDXMXfh7" width="500">

Let's now analyze the image with the lady and wine:

<img src="https://drive.google.com/uc?id=1VLUwgA3uxav88H6Q6QGE8bh_DnDGbdck" width="500"> <img src="https://drive.google.com/uc?id=1IFtdhU9mxew7XeavTo5ZbNsD_7VWFNDi" width="500">

In this case the two model seems to incorporate in the prediction all the bottles of wine.

Last but not least let's see an example with a correct prediction, in this casee the referring expression is: "a younf woman wearing a green shirt eating a piece of pizza with her legs up"

<img src="https://drive.google.com/uc?id=1IbGlrGH0NcajYogXJ_q8Wz6CrT8UUCbj" width="500"> <img src="https://drive.google.com/uc?id=17DmSJL2W2h5ZvZSttvRYNUsCVlmYC7hf" width="500">

In this case, there is no ambiguity in the sentence in respect to the image and so the model is able to correctly find the bounding box.

### *Conclusion*

As we have seen from the examples the two models seems to behave in the same way and they share the same weakness, of course the bigger model is able to capture more information and have a better accuracy.


In [ ]:
class BOHModule(nn.Module):
    """
    This module basically applies cross attention using text as query and visual as key and value
    then applies a MLP on the output of the cross attention
    """
    def __init__(self, shared_dim=128, dropout=0.1, bias=True):
        super(BOHModule, self).__init__()

        self.lnt = nn.LayerNorm(shared_dim)
        self.lnv = nn.LayerNorm(shared_dim)

        self.mhca = nn.MultiheadAttention(embed_dim=shared_dim, num_heads=8, dropout=0.1)

        self.ln2 = nn.LayerNorm(shared_dim)
        self.mlp = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(shared_dim, 2 * shared_dim, bias=bias),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(2 * shared_dim, shared_dim, bias=bias),
        )

    def forward(self, t, v):
        # t: text, v: visual -> t is used as query, v as key and value
        t = t + self.mhca(self.lnt(t), self.lnv(v), self.lnv(v), need_weights=False)[0]
        t = t + self.mlp(self.ln2(t))
        return t

class BOH(nn.Module):
    def __init__(self, clip_model, shared_space_dim=128, num_fusion=4, dropout=0.2, use2loc=False, device='mps'):
        """
        Implementation of the BOH model
        :param clip_model: the CLIP model
        :param shared_space_dim: the dimension of the shared space used in all the modules
        :param num_fusion: the number of BOH modules
        :param dropout: dropout rate
        :param use2loc: tells if to use 2 localization tokens or 1
        :param device: device
        """
        super(BOH, self).__init__()
        self.clip_model = clip_model.requires_grad_(False).to(device)
        self.device = device
        self.num_fusion = num_fusion

        self.visual_reduction = nn.Linear(768, shared_space_dim)
        self.text_reduction = nn.Linear(512, shared_space_dim)

        self.fusion_adapters = nn.ModuleList([
            BOHModule(shared_dim=shared_space_dim, dropout=dropout)
            for _ in range(num_fusion)
        ])


        # localization token ->   to zero
        self.loc = nn.Parameter(torch.zeros((1, 1, shared_space_dim)).to(device))

        self.regression = nn.Sequential(
            nn.Linear(shared_space_dim, shared_space_dim, bias=False),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(shared_space_dim, 4, bias=False),
            nn.Sigmoid()
        )


    def before_visual_transformer(self, x):
        """
        Code copied from the clip implementation in order to preprocess the image ready to be fed into the transformer.
        :param x: image tensor
        :return: embedding tensor for the transformer
        """
        x = self.clip_model.visual.conv1(x)  # shape = [*, width, grid, grid]
        x = x.reshape(x.shape[0], x.shape[1], -1)  # shape = [*, width, grid ** 2]
        x = x.permute(0, 2, 1)  # shape = [*, grid ** 2, width]
        x = torch.cat([
            self.clip_model.visual.class_embedding.to(x.dtype)
            + torch.zeros(x.shape[0], 1, x.shape[-1], dtype=x.dtype, device=x.device),
            x
        ], dim=1)  # shape = [*, grid ** 2 + 1, width]

        x = x + self.clip_model.visual.positional_embedding.to(x.dtype)
        x = self.clip_model.visual.ln_pre(x)
        x = x.permute(1, 0, 2)  # NLD -> LND
        return x

    def before_text_transformer(self, x):
        """
        Code copied from the clip implementation in order to preprocess the text ready to be fed into the transformer.
        :param x: text tokens tensor
        :return: the embedding tensor for the transformer
        """
        x = self.clip_model.token_embedding(x).type(self.clip_model.dtype)  # [batch_size, n_ctx, d_model]
        x = x + self.clip_model.positional_embedding.type(self.clip_model.dtype)
        x = x.permute(1, 0, 2)  # NLD -> LND
        return x

    def forward(self, image, text):
        v = self.before_visual_transformer(image)  # [197, batch, 768] -> [CLS + PATCHES, BATCH, HIDDEN_SIZE]
        t = self.before_text_transformer(text)  # [77, batch, 512] -> [SOS + TOKENS + EOS, BATCH, HIDDEN_SIZE]

        # assuming transformers have both 12 blocks
        num_normal_blocks = 12 - self.num_fusion + 1

        # run normal transformer blocks
        for i in range(num_normal_blocks):
            v = self.clip_model.visual.transformer.resblocks[i](v)
            t = self.clip_model.transformer.resblocks[i](t)

        # create the LOC token
        loc = self.loc.expand(-1, v.shape[1], -1)  # [1, BATCH, HIDDEN_SIZE]
        t_hat = torch.cat([loc, self.text_reduction(t)], dim=0)  # [1 + 77, BATCH, HIDDEN_SIZE]

        for i in range(self.num_fusion - 1):
            # add loc token to t
            t_hat = self.fusion_adapters[i](t_hat, self.visual_reduction(v)) # [1 + 77, BATCH, HIDDEN_SIZE]
            # continue res blocks
            t = self.clip_model.transformer.resblocks[num_normal_blocks + i](t)
            v = self.clip_model.visual.transformer.resblocks[num_normal_blocks + i](v)
            # add residual connection for my adapter, loc token not considered
            t_hat[1:] = t_hat[1:] + self.text_reduction(t)

        # last adapter
        v = self.visual_reduction(v)
        t_hat = self.fusion_adapters[-1](t_hat, v)
        loc = t_hat[0:1, :, :]  # [LOC, BATCH, HIDDEN_SIZE]

        return self.regression(loc.squeeze())


## BOH with 256 shared space

In [ ]:
net = BOH(clip_model, shared_space_dim=256, dropout=0.3, num_fusion=8, device=DEVICE).to(DEVICE)

experiment_name = "boh_256_8"

last_epoch, net = load_last_checkpoint(net, experiment_name, device=DEVICE)

criterion = SIoULoss()

# Uncomment to train
# optimizer = torch.optim.AdamW(net.parameters(), lr=0.0001, weight_decay=0.02)
# train_evaluate(net, optimizer, criterion, epochs=20,
#               experiment_name=experiment_name,
#               train_dataloader=train_dataloader,
#               val_dataloader=val_dataloader,
#               test_dataloader=test_dataloader,
#               last_epoch=last_epoch,
#               tokenize = clip.tokenize,
#               device=DEVICE)

# Run only the test
test_loss, test_acc, test_miou = test_step(net, test_dataloader, criterion, clip.tokenize, DEVICE)
print('\tTest loss {:.5f}, Acc: {:.5f}, mIoU: {:.5f}'.format(test_loss, test_acc, test_miou))

Loading weights for experiment boh_256_8 from epoch 20


Loss: 0.8692959200951361, Acc: 0.12903225806451613, mIoU: 0.2427009642124176: 100%|██████████| 79/79 [02:13<00:00,  1.70s/it]

	Test loss 0.79145, Acc: 0.18276, mIoU: 0.29974


## BOH with 768 shared space

In [ ]:
net = BOH(clip_model, shared_space_dim=768, dropout=0.3, num_fusion=8, device=DEVICE).to(DEVICE)

experiment_name = "boh_768_8"

last_epoch, net = load_last_checkpoint(net, experiment_name, device=DEVICE)

criterion = SIoULoss()

# Uncomment to train
# optimizer = torch.optim.AdamW(net.parameters(), lr=0.0001, weight_decay=0.02)
# train_evaluate(net, optimizer, criterion, epochs=20,
#               experiment_name=experiment_name,
#               train_dataloader=train_dataloader,
#               val_dataloader=val_dataloader,
#               test_dataloader=test_dataloader,
#               last_epoch=last_epoch,
#               tokenize = clip.tokenize,
#               device=DEVICE)

# Run only the test
test_loss, test_acc, test_miou = test_step(net, test_dataloader, criterion, clip.tokenize, DEVICE)
print('\tTest loss {:.5f}, Acc: {:.5f}, mIoU: {:.5f}'.format(test_loss, test_acc, test_miou))

Loading weights for experiment boh_768_8 from epoch 38


Loss: 0.8471358514601185, Acc: 0.22580645161290322, mIoU: 0.26215916872024536: 100%|██████████| 79/79 [02:26<00:00,  1.86s/it]

	Test loss 0.79089, Acc: 0.19550, mIoU: 0.30200


# BOHv2

# *Introduction*
The BOHV2 model can be seen as an extension or evolution of the BOH model. While both models utilize the same fundamental idea of using attention mechanisms to relate elements of input text with parts of an input image and the novel LOC token, there are key differences in their architectural designs.

# *Idea*
While the BOH model applies cross-attention using text as the query and the visual representation as key and value, the BOHV2 model goes a step further and applies cross-attention in both directions.

Both models use "fusion adapters" to apply attention between the text and visual modalities. However, in the BOHV2 model, there's an extension called "CrossGuidedAttentionModule". This module applies cross-attention in both directions, but keeping the tokens separeted, i.e., using text as a query and visual as key-value, and vice versa.

The main idea is to keep the two fluxes separate in order not to influence each other and obtain cleaner results.

# *Architecture*

In the original BOH model, cross-attention is applied in one direction, with the textual part acting as the query and the visual part as the key-value pair.

CGA Modules :In contrast, BOHv2's Cross-Guided Attention Module applies cross-attention in both directions: text as the query and visual as the key-value pair, and vice versa. This allows the model to consider both visual-to-textual and textual-to-visual relationships.

Fusion Modules: The BOHv2 model introduces a series of fusion adapters, which are instances of the Cross-Guided Attention Module. These modules are applied to the transformed visual and textual features, enabling the model to better integrate information from both modalities.

Regression Layer: In both models, the final output is produced by a regression layer. However, it's important to note that in the original BOH model, this regression layer operates on the LOC token, while in BOHv2, it operates on the concatenation of the two LOC token.

The following scheme illustrates the architecture:
![picture](https://drive.google.com/uc?id=1xpxAHfDlGobje2h9-gbRZr4VAqFXTeHl)


## *Implementations*

As for the preious method, we trained this model in two configurations, using shared spaces of size 256 and 512, respectively. And for both models, we utilized the same number of cross-attention modules, set at 9.


# *Training results*

#### *BOHv2 256*
As seen from the graphs the results were very good right from the start.

![picture](https://drive.google.com/uc?id=1gDvNhbdvssO0oKmw4ecJgorn3svJK7_S)

One thing noticed right away, and confirmed through some examples (illustrated  below) is that the model acquired the possibility to actually develop spatial attention in order to find enetities in the image.
The performance of the model was also very good, almost reaching baseline level accuracy.

We must consider that the a lot of bounding boxes gets close to the target anyway but don't count as positive prediction beacuse its IoU was slightly smaller than 0.5.

That said this model can achieve an accuracy of 42% and mIoU of 0.43 on the test set.


#### *BOHv2 512*
As expected with an higher we were able to achieve better results:
![picture](https://drive.google.com/uc?id=1l1OgDgodOhDyIqosmhJrEYbkXPbZILG9)
As we can infer from the curve, there is still some margin of improvement, and with some epochs more we could reach some very interesting results.

#### *Comparison*

![picture](https://drive.google.com/uc?id=1i8o-XAfp-VHf6cN1laIYVJCf8T5v_yHs)

Let's have a look now on some examples, the first image is always referred to the small model and the second to the bigger one.

text : "the woman on the right"

<img src="https://drive.google.com/uc?id=1N1Yo81YcZGNGuFWXCWhmgPPh5UTaRPCf" width="500"> <img src="https://drive.google.com/uc?id=1x1YyG7a7UxA2wnRNMe7hud57cXfaPnSD" width="500">

text : "small green vase on the left with a flower in it"

<img src="https://drive.google.com/uc?id=12U7h2omE8kamnkzxvIr_Z9nNj2fazQQZ" width="500"> <img src="https://drive.google.com/uc?id=1azF_vmBq6n-uXW9HHptkgzuUbAZHFWqZ" width="500">

text : "a lady pouring wine in a glass"

<img src="https://drive.google.com/uc?id=1H3jpYAyVICooErdSAE-65QqNLGbUXU3o" width="500"> <img src="https://drive.google.com/uc?id=1lTAoChHiVNf06pJCH7VxsYyTgJVP6eLO" width="500">

text : "a young woman wearing a green shirt eating a piece of pizza with her legs up"

<img src="https://drive.google.com/uc?id=1JYw2PsQSDmcNunJtSFDKyT3rRMMv5Dgo" width="500"> <img src="https://drive.google.com/uc?id=1YxmIfCY1Z8aXxc_pPvZsBtrVJab5UODa" width="500">

text : "elephant facing towards a red cement wall"

<img src="https://drive.google.com/uc?id=1zXPve7tGmUwflFOxQjfQR3fVIW9JNkk1" width="500"> <img src="https://drive.google.com/uc?id=187x7QW4QdfaFS91Gc9Sl39vqLwjM4sfk" width="500">


As seen in the examples the models has still some problems on complex shapes. It is not able, for example to recognize the leg as part of the woman eating the pizza, or it does not see the elephant facing the wall.
Another observation on the image is that the bigger model can better enclose the object into the bounging box.


In [ ]:
class CrossGuidedAttentionModule(nn.Module):
    """
    This module applies cross attention in both directions:
    text as query and visual as key and value, and vice versa.
    Then it applies an MLP on the output of the cross attention.
    """
    def __init__(self, shared_dim=256, dropout=0.1):
        super(CrossGuidedAttentionModule, self).__init__()

        self.lnt = nn.LayerNorm(shared_dim)
        self.lnv = nn.LayerNorm(shared_dim)

        self.mhca_tv = nn.MultiheadAttention(embed_dim=shared_dim, num_heads=8, dropout=0.1)  # Text to Visual
        self.mhca_vt = nn.MultiheadAttention(embed_dim=shared_dim, num_heads=8, dropout=0.1)  # Visual to Text

        self.ln2 = nn.LayerNorm(shared_dim)
        self.mlp = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(shared_dim, 2 * shared_dim, ),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(2 * shared_dim, shared_dim),
        )

    def forward(self, t, v):
        # t: text, v: visual -> t is used as query, v as key and value and vice versa
        t = t + self.mhca_tv(self.lnt(t), self.lnv(v), self.lnv(v), need_weights=False)[0]
        v = v + self.mhca_vt(self.lnv(v), self.lnt(t), self.lnt(t), need_weights=False)[0]

        t = t + self.mlp(self.ln2(t))
        v = v + self.mlp(self.ln2(v))

        return t, v


class BOHv2Model(nn.Module):
    def __init__(self, clip_model, shared_space_dim=128, num_fusion=4, dropout=0.2, device='cpu'):
        super(BOHv2Model, self).__init__()
        self.clip_model = clip_model.requires_grad_(False).to(device)
        self.device = device
        self.num_fusion = num_fusion

        self.visual_reduction = nn.Linear(768, shared_space_dim)  # Adjust this if your visual input size is not 768
        self.text_reduction = nn.Linear(512, shared_space_dim)  # Adjusted this to match your text input size

        self.loc_token_v = nn.Parameter(torch.randn(1, 1, shared_space_dim))
        self.loc_token_t = nn.Parameter(torch.randn(1, 1, shared_space_dim))

        self.fusion_adapters = nn.ModuleList([
            CrossGuidedAttentionModule(shared_dim=shared_space_dim, dropout=dropout)
            for _ in range(num_fusion)
        ])

        self.regression = nn.Sequential(
            nn.Linear(shared_space_dim*2, shared_space_dim, bias=False),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(shared_space_dim, 4, bias=False),
            nn.Sigmoid()
        )

    def before_visual_transformer(self, x):
        x = self.clip_model.visual.conv1(x)
        x = x.reshape(x.shape[0], x.shape[1], -1)
        x = x.permute(0, 2, 1)
        x = torch.cat([
            self.clip_model.visual.class_embedding.to(x.dtype)
            + torch.zeros(x.shape[0], 1, x.shape[-1], dtype=x.dtype, device=x.device),
            x
        ], dim=1)

        x = x + self.clip_model.visual.positional_embedding.to(x.dtype)
        x = self.clip_model.visual.ln_pre(x)
        x = x.permute(1, 0, 2)
        return x

    def before_text_transformer(self, x):
        x = self.clip_model.token_embedding(x).type(self.clip_model.dtype)
        x = x + self.clip_model.positional_embedding.type(self.clip_model.dtype)
        x = x.permute(1, 0, 2)
        return x

    def forward(self, image, text):
        v = self.before_visual_transformer(image)
        t = self.before_text_transformer(text)

        num_normal_blocks = 12 - self.num_fusion + 1

        for i in range(num_normal_blocks):
            v = self.clip_model.visual.transformer.resblocks[i](v)
            t = self.clip_model.transformer.resblocks[i](t)

        loc_v = self.loc_token_v.expand(-1, v.shape[1], -1)
        v_reduced = torch.cat([loc_v, self.visual_reduction(v)], dim=0)

        loc_t = self.loc_token_t.expand(-1, t.shape[1], -1)
        t_reduced = torch.cat([loc_t, self.text_reduction(t)], dim=0)

        for i in range(self.num_fusion - 1):
            t_reduced, v_reduced = self.fusion_adapters[i](t_reduced, v_reduced)
            v = self.clip_model.visual.transformer.resblocks[i + num_normal_blocks](v)
            t = self.clip_model.transformer.resblocks[i + num_normal_blocks](t)
            t_reduced[1:, :, :] = t_reduced[1:, :, :] + self.text_reduction(t)
            v_reduced[1:, :, :] = v_reduced[1:, :, :] + self.visual_reduction(v)

        t_reduced, v_reduced = self.fusion_adapters[i](t_reduced, v_reduced)

        # t = t.mean(dim=0)  # Mean pooling across the sequence dimension
        v = v_reduced[0]
        t = t_reduced[0]

        # cat and flat
        x = torch.cat([t, v], dim=-1)
        x = x.view(x.shape[0], -1)
        return self.regression(x)

## BOHv2 with 256 shared space

In [ ]:
net = BOHv2Model(clip_model, shared_space_dim=256, dropout=0.3, num_fusion=6, device=DEVICE).to(DEVICE)

experiment_name = "boh_v2_256_8"

last_epoch, net = load_last_checkpoint(net, experiment_name, device=DEVICE)

criterion = SIoULoss()

# Uncomment to train
# optimizer = torch.optim.AdamW(net.parameters(), lr=0.0001, weight_decay=0.02)
# train_evaluate(net, optimizer, criterion, epochs=20,
#               experiment_name=experiment_name,
#               train_dataloader=train_dataloader,
#               val_dataloader=val_dataloader,
#               test_dataloader=test_dataloader,
#               last_epoch=last_epoch,
#               tokenize = clip.tokenize,
#               device=DEVICE)

# Run only the test
test_loss, test_acc, test_miou = test_step(net, test_dataloader, criterion, clip.tokenize, DEVICE)
print('\tTest loss {:.5f}, Acc: {:.5f}, mIoU: {:.5f}'.format(test_loss, test_acc, test_miou))

Loading weights for experiment boh_v2_256_8 from epoch 28


Loss: 0.6718753691642515, Acc: 0.22580645161290322, mIoU: 0.3994470536708832: 100%|██████████| 79/79 [02:13<00:00,  1.69s/it]

	Test loss 0.63496, Acc: 0.42047, mIoU: 0.42838


## BOHv2 with 512 shared space

In [ ]:
net = BOHv2Model(clip_model, shared_space_dim=512, dropout=0.3, num_fusion=6, device=DEVICE).to(DEVICE)

experiment_name = "boh_v2_512"

last_epoch, net = load_last_checkpoint(net, experiment_name, device=DEVICE)

criterion = SIoULoss()

# Uncomment to train
# optimizer = torch.optim.AdamW(net.parameters(), lr=0.0001, weight_decay=0.02)
# train_evaluate(net, optimizer, criterion, epochs=20,
#               experiment_name=experiment_name,
#               train_dataloader=train_dataloader,
#               val_dataloader=val_dataloader,
#               test_dataloader=test_dataloader,
#               last_epoch=last_epoch,
#               tokenize = clip.tokenize,
#               device=DEVICE)

# Run only the test
test_loss, test_acc, test_miou = test_step(net, test_dataloader, criterion, clip.tokenize, DEVICE)
print('\tTest loss {:.5f}, Acc: {:.5f}, mIoU: {:.5f}'.format(test_loss, test_acc, test_miou))

Loading weights for experiment boh_v2_512 from epoch 39


Loss: 0.6258601527060231, Acc: 0.3225806451612903, mIoU: 0.4367911219596863: 100%|██████████| 79/79 [02:16<00:00,  1.73s/it]

	Test loss 0.62278, Acc: 0.45192, mIoU: 0.44084


# *Conclusion*

In wrapping up our project, we opted for a linear approach. Initially, we set up the baseline as suggested in the assignment. We then explored a training-free method, specifically the recursive region model. From there, we decided to switch into solutions involving training. Cross-attention attracted our attention so we decided to go in that way. Our first step in this direction was to base our work on a research paper with promising results, which allowed us to gain a more comprehensive understanding of how cross-attention and transformers operates.

Following this exploration, we proposed a novel concept: the LOC (Localization) token. It is added to the token or the patches dimension, its purpose is to identify the location of the referred object.

At first, we experimented with this token within a simple cross-attention module. This module computed cross-attention exclusively using textual features as the query and using visual aspects as the key and value. After this, we tried to apply the concept in both directions.

## *Training hyperparameters*

For all our models that required training, we implemented specific strategies to prevent overfitting. We always used the AdamW optimizer with weight decay to penalize parameters with excessively high values. The weight decay value was set at 0.02. We also incorporated dropout as another countermeasure against overfitting.

In almost all cases, we adopted the same training strategy across models: we began with a learning rate of 1e-04 and progressively reduced this value after 20 epochs.

Moreover, we saved the weights of our models at every epoch in order to be able to get back the best model when the training process was overfitting.


## *Models comparison*


| Model            |Hidden Size | Model Size | Accuracy | mIoU   |
|------------------|------------|------------|----------|--------|
| BASELINE         | -          | -          | **0.53** |**0.52**|
| RECURSIVE REGION | -          | -          | 0.11     | 0.25   |
| BOH              | 256        | 4M         | 0.18     | 0.29   |
| BOH              | 768        | 39M        | 0.20     | 0.30   |
| BOH v2           | 256        | 5M         | 0.42     | 0.43   |
| BOH v2           | 512        | 20M        | 0.45     | 0.44   |


We already discussed in relative chapters weakness and strenght of every model, so here we will do just a quantitative anlysis.
As we can see the best model is still the baseline, but we are convinced that with some adjustments we may do better.
Our BOHv2 can almost always find the right area where to focus but struggles on having high IoU and a lot of prediction are missed out because they have IoU smaller but very close to 0.5

A note on the model size: it is not considering the size of the CLIP model.


Here we have all the training curves of our trained models:

![picture](https://drive.google.com/uc?id=1quX_ZwQVNTCLmAZSHy_pspyeDqwQc-Vl)

## *Findings*

As we can clearly see from the results we did not did better than the baseline because as said in the relative chapter it is using yolo that is trained on the RefCOCO datasets and thus it gives almost always very good proposals.

However, reagrding our results we can conlcude that the intuition we had was correct and that we can use a LOC token in order to correctly find the position of the object in the image. Note that the BOHv2 model we trained is a very small model and in comparison with the other models it performed way way better. This can suggest us that with a bigger and maybe more sophisticated model we may achieve even better results.

To conclude we want to remark again our finding that the strategy to use a LOC token in that way is a promising idea and opens another question on how much powerful are that kind of tokens. Can be this kind of tokens being used even for other scenarios? If yes, this may open new ways to handle potentially a wide range of new tasks.


## *Limitations and further works*

Throughout this project, we encountered numerous challenges at nearly every stage of the process, from group formation to the implementation of our models. However, the main issue we deal with was the limited resources at our disposal. With more resources, we could have analyzed better our models and experimented with a broader range of hyperparameters, such as the number of cross-attention modules used or different model sizes, to draw more robust conclusions.

Nonetheless, we know that the purpose of this project was not to train large models. And we are pleased with our results, we hope that these findings may contribute a small piece to the larger puzzle of research.

Further works should analyze better hyperparameters like the number of cross attention modules. We would try to add self attention on the top of the lasts cross attention modules to see if it can further improve performance like several works in the literature does.


## *Last minute found problems*

Reviewing our project we found that we were splitting the train dataset into training and validation sets, when we already had a validation split into the dataset. This is a problem because at the end of the day we used only the 80% of data available to train our models.


This problem introduced another problem with this specific line of code:
```
train_dataset, validation_dataset = torch.utils.data.random_split(train, [math.floor(len(train) * 0.8), math.ceil(len(train) * 0.2)])
```
Doing that we are random splitting our dataset, that may seem correct but given the fact that sometimes we were restarting the training procedure and loading the datasets again we then trained our models with different training and validation set. Moreover, the validation set in subsequent runs was composed of samples that were in the training set and so the validation loss was no more an indicator of overfitting.
This can even explain the strange curves on the validation data with some strange big boost on better values, corresponding to when we resumed the training.